<a href="https://colab.research.google.com/github/geoalinho/cascata-do-salso/blob/main/Cascata%20do%20Salso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import ee
import geemap
import requests
import geopandas as gpd

from google.colab import files
from pathlib import Path

In [6]:
ee.Authenticate()
ee.Initialize(project='ee-alanrgeo')

In [7]:
class AreaOfInterest:
  def __init__():
    pass

salso_cascade_point = ee.Geometry.Point([-53.448807548002016, -30.569416210967997]  )
salso_cascade_area = salso_cascade_point.buffer(1000)
salso_cascade_area = salso_cascade_area.bounds()

In [8]:
class DigitalElevationModel:
  def __init__(self, dem_id, *, band=None, bounds=None):
    self.id = dem_id

    if band:
      self.image = ee.Image(self.id).select(band)
    else:
      self.image = ee.Image(self.id)

    if bounds:
      self.image = self.image.clip(bounds)

  def hillshade(self, azimuth, angle_altitude):
    return ee.Terrain.hillshade(self.image, azimuth, angle_altitude)

  def aspect(self):
    return ee.Terrain.aspect(self.image)

In [9]:
class Palettes:
  @staticmethod
  def green_yellow_red():
    return ['green', 'yellowgreen', 'yellow', 'goldenrod', 'orange', 'red']

  @staticmethod
  def blues():
    return ['white', 'lightblue', 'blue']

  @staticmethod
  def rainbow():
    return ['darkblue', 'blue', 'lightblue', 'green', 'yellow', 'orange', 'red', 'white']

In [10]:
class VisualizationParameters:
  @staticmethod
  def min_and_max(image, band, *, palette=None):
    if(palette == None):
      palette = Palettes.rainbow()

    min = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.min(), geometry=salso_cascade_area, scale=10).get(band))
    max = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.max(), geometry=salso_cascade_area, scale=10).get(band))
    return {'min': min,
            'max': max,
            'palette': palette}

  def cumulative_cut(image, band, *, palette=None, min=2, max=98):
    if(palette == None):
      palette = Palettes.rainbow()

    min = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.percentile([min]), geometry=salso_cascade_area, scale=10).get(band))
    max = ee.Number(image.select(band).reduceRegion(reducer=ee.Reducer.percentile([max]), geometry=salso_cascade_area, scale=10).get(band))
    return {'min': min,
            'max': max,
            'palette': palette}

In [ ]:
class ImageDownloadParams:
    def __init__(self, *, name: str, bands: list, crs: str, scale: int, region: ee.Geometry, format: str):
        self.name = name
        self.bands = bands
        self.crs = crs
        self.scale = scale
        self.region = region
        self.format = format

In [43]:
class Deleter:
    @staticmethod
    def clear_colab_folder(colab_folder: Path):
        for f in colab_folder.iterdir():
            f = Path(f)
            if f.is_file():
                f.unlink()

class Downloader:
  @staticmethod
  def ee_image_to_colab(colab_path: Path, image: ee.Image, file_name: str, params: ImageDownloadParams):
    url = image.getDownloadURL(params.__dict__)
    response = requests.get(url)
    file_path = colab_path / file_name
    file_path.write_bytes(response.content)

class Uploader:
    @staticmethod
    def desktop_shapefile_to_colab(colab_path: Path):
        needed_extensions = {'.cpg': False,
                             '.dbf': True,
                             '.prj': True,
                             '.shp': True,
                             '.shx': True,
                             '.qix': False,
                             '.qmd': False}

        files_to_upload = files.upload()
        shapefile_path = None

        for f in files_to_upload:
            f = Path(f)
            file_extension = f.suffix

            if not needed_extensions[file_extension]:
                f.unlink()

Uploader.desktop_shapefile_to_colab(Path('/content/'))

Saving pontos_coleta_campo_24-04-2025.cpg to pontos_coleta_campo_24-04-2025.cpg
Saving pontos_coleta_campo_24-04-2025.dbf to pontos_coleta_campo_24-04-2025.dbf
Saving pontos_coleta_campo_24-04-2025.prj to pontos_coleta_campo_24-04-2025.prj
Saving pontos_coleta_campo_24-04-2025.qix to pontos_coleta_campo_24-04-2025.qix
Saving pontos_coleta_campo_24-04-2025.qmd to pontos_coleta_campo_24-04-2025.qmd
Saving pontos_coleta_campo_24-04-2025.shp to pontos_coleta_campo_24-04-2025.shp
Saving pontos_coleta_campo_24-04-2025.shx to pontos_coleta_campo_24-04-2025.shx


In [12]:
# SENTINEL-2 COLLECTION
sentinel_2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
salso_cascade_sentinel_2 = sentinel_2.filterBounds(salso_cascade_area)
salso_cascade_sentinel_2_2024 = salso_cascade_sentinel_2.filterDate('2024-01-01', '2024-12-31')
salso_cascade_sentinel_2_2024_cloud_free = salso_cascade_sentinel_2_2024.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))

# SENTINEL-2 RGB MOSAIC
salso_cascade_sentinel_2_rgb = salso_cascade_sentinel_2_2024_cloud_free.select(['TCI_R', 'TCI_G', 'TCI_B'])
salso_cascade_sentinel_2_rgb_mosaic = salso_cascade_sentinel_2_rgb.mosaic()
salso_cascade_sentinel_2_rgb_mosaic = salso_cascade_sentinel_2_rgb_mosaic.clip(salso_cascade_area)

In [13]:
# SENTINEL-2 NDMI MOSAIC
salso_cascade_sentinel_2_2024_ndmi = salso_cascade_sentinel_2_2024_cloud_free.select(['B8', 'B11']).map(lambda image: image.normalizedDifference(['B8', 'B11']).rename('NDMI'))
salso_cascade_sentinel_2_2024_ndmi_mosaic = salso_cascade_sentinel_2_2024_ndmi.mosaic()
salso_cascade_sentinel_2_2024_ndmi_mosaic = salso_cascade_sentinel_2_2024_ndmi_mosaic.clip(salso_cascade_area)

# SENTINEL-2 NDWI MOSAIC
salso_cascade_sentinel_2_2024_ndwi = salso_cascade_sentinel_2_2024_cloud_free.select(['B3', 'B8']).map(lambda image: image.normalizedDifference(['B3', 'B8']).rename('NDWI'))
salso_cascade_sentinel_2_2024_ndwi_mosaic = salso_cascade_sentinel_2_2024_ndwi.mosaic()
salso_cascade_sentinel_2_2024_ndwi_mosaic = salso_cascade_sentinel_2_2024_ndwi_mosaic.clip(salso_cascade_area)

In [14]:
# LANDSAT-8 COLLECTION
landsat_8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
salso_cascade_landsat_8 = landsat_8.filterBounds(salso_cascade_area)
salso_cascade_landsat_8_2024_cloud_free = salso_cascade_landsat_8.filter(ee.Filter.lt('CLOUD_COVER', 10))

# LANDSAT-8 SALSO CASCADE RGB MOSAIC
salso_cascade_landsat_8_rgb = salso_cascade_landsat_8_2024_cloud_free.select(['SR_B4', 'SR_B3', 'SR_B2'])
salso_cascade_landsat_8_rgb_mosaic = salso_cascade_landsat_8_rgb.mosaic()
salso_cascade_landsat_8_rgb_mosaic = salso_cascade_landsat_8_rgb_mosaic.clip(salso_cascade_area)

In [15]:
# LANDSAT-8 SALSO CASCADE TIRS MOSAIC
salso_cascade_landsat_8_tirs = salso_cascade_landsat_8_2024_cloud_free.select(['ST_TRAD'])
salso_cascade_landsat_8_tirs_mosaic = salso_cascade_landsat_8_tirs.mosaic()
salso_cascade_landsat_8_tirs_mosaic = salso_cascade_landsat_8_tirs_mosaic.clip(salso_cascade_area).reproject('EPSG:4326', scale=1)

In [16]:
# LANDSAT-8 SURFACE TEMPERATURE
constant_1 = ee.Image.constant(0.00341802)
constant_2 = ee.Image.constant(149)
constant_3 = ee.Image.constant(273.15)

salso_cascade_landsat_8_st_kelvin = ee.Image(salso_cascade_landsat_8_tirs_mosaic).multiply(constant_1).add(constant_2)

salso_cascade_landsat_8_st_celsius = salso_cascade_landsat_8_st_kelvin.subtract(constant_3)

In [17]:
dem = DigitalElevationModel('NASA/NASADEM_HGT/001', band='elevation', bounds=salso_cascade_area)
salso_cascade_dem = dem.image
salso_cascade_dem_hillshade = dem.hillshade(-329.88, 40.78)
salso_cascade_dem_aspect = dem.aspect()

In [45]:
dem_url = salso_cascade_dem.getDownloadURL(params)

download_params = ImageDownloadParams(name='dem',
                                      bands=['elevation'],
                                      crs='EPSG:4326',
                                      scale=10,
                                      region=salso_cascade_area,
                                      format='GEO_TIFF')

# Downloader.ee_image_to_colab(Path('/content/'), salso_cascade_dem, 'dem.tif', download_params)

In [48]:
map = geemap.Map()
map.add_basemap('SATELLITE')
map.centerObject(salso_cascade_point, 15)

points = gpd.read_file('/content/pontos_coleta_campo_24-04-2025.shp')

# map.addLayer(salso_cascade_landsat_8_tirs_mosaic,
#              VisualizationParameters.min_and_max(salso_cascade_landsat_8_tirs_mosaic,
#                                                  'ST_TRAD',
#                                                  palette=Palettes.rainbow()),
#              name='Landsat 9 TIRS')

# map.addLayer(salso_cascade_landsat_8_tirs_mosaic,
#              VisualizationParameters.cumulative_cut(salso_cascade_landsat_8_tirs_mosaic,
#                                                     'ST_TRAD',
#                                                     palette=Palettes.rainbow(),
#                                                     min=2,
#                                                     max=97),
#              name='Landsat 9 TIRS - cumulative_cut')

map.addLayer(salso_cascade_landsat_8_st_celsius,
            #  VisualizationParameters.cumulative_cut(salso_cascade_landsat_8_st_celsius,
            #                                         'ST_TRAD',
            #                                         palette=Palettes.rainbow(),
            #                                         min=2,
            #                                         max=97),
             {'palette': Palettes.rainbow(), 'min': -91, 'max':-89},
             name='Landsat 9 Celsius - cumulative_cut',
             shown=False)
map.add_gdf(points, layer_name='Pontos de Coleta')

map

Map(center=[-30.569416210967997, -53.448807548002016], controls=(WidgetControl(options=['position', 'transpare…